In [ ]:
from mstr_robotics.json_analyser import parse_json

from mstr_robotics.navigation import mstr_objects
from mstr_robotics.report import rep, prompts
from mstr_robotics.redis_db import  redis_bi_analysis
import json 
from mstr_robotics.json_analyser import parse_json
from mstr_robotics._helper import msic
from mstr_robotics.user_RAG import keyword_processor,  perplexity

u_keyword_processor=keyword_processor()
os_mcp_folder_str="D:\\shared_drive\\mstr_output\\MCP_data"

user_path="D:\\shared_drive\\Python\\mstr_robotics\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)

i_prompts=prompts()
i_rep=rep()
u_perplexity=perplexity()
i_mstr_objects=mstr_objects()
i_msic=msic()
i_parse_json=parse_json()

def open_redis_connection(redis_con_d): 
        i_redis_bi_analysis=redis_bi_analysis( 
                host=redis_con_d["host"],
                port=redis_con_d["port"],
                password=redis_con_d["password"],  # Redis Cloud requires auth
                username=redis_con_d["username"],  # Redis Cloud requires username
                decode_responses=redis_con_d["decode_responses"]
                                )
        return i_redis_bi_analysis   

def get_dependency(i_redis_bi_analysis,object_j,target_keys_l=["id", "key", "objectId"]):
    all_obj_def_d_l=[]
    #all_obj_def_d_l.extend(dossier_j)

    child_obj_keys_d_l=i_parse_json.extract_specific_key_value_pairs(data=object_j, target_keys=target_keys_l, 
                                    prefix="root", all_pairs=None, parent_name="")
    child_obj_keys_l=i_msic.get_key_form_dict_l(dict_l=child_obj_keys_d_l, key="value")

    all_keys_l=child_obj_keys_l
    fetch_keys_l=all_keys_l
    while len(fetch_keys_l)>0:
        child_obj_d_l=i_redis_bi_analysis.fetch_key_list(key_l=fetch_keys_l)
        objects_d_l=check_success_status(child_obj_d_l)
        all_obj_def_d_l.extend(objects_d_l)
        
        child_obj_keys_d_l=i_parse_json.extract_specific_key_value_pairs(data=objects_d_l, target_keys=target_keys_l)
        child_obj_keys_l=i_msic.get_key_form_dict_l(dict_l=child_obj_keys_d_l, key="value")
        fetch_keys_l= list(set(child_obj_keys_l) - set(all_keys_l))
        all_keys_l.extend(child_obj_keys_l)

    return all_obj_def_d_l

def check_success_status(child_obj_d_l):
    object_id_l=[]
    for obj in child_obj_d_l:
        if obj["value"]["status"]=="success":
            if len(obj["value"]["value"])>0:
                object_id_l.append(obj["value"]["value"].copy())


    return object_id_l

redis_env_d={"mstr_dev":{
        "host":"redis-14995.c56.east-us.azure.redns.redis-cloud.com",
        "port":14995,
        "password":"KSvQA45EE8ZEJ3xcFvus96927CW56D8u",  
        "username":"default",  
        "decode_responses":True,
        "project_id":"B7CA92F04B9FAE8D941C3E9B7E0CD754",
        "object_id":"98EB31B54122FFB738E6E08A2F29421A"
},
        "mstr_test":{
        "host":"redis-13092.c282.east-us-mz.azure.redns.redis-cloud.com",
        "port":13092,
        "password":"pzSq7pQas9TFd6gMhcypYC70HRdyyvRv",
        "username":"default",
        "decode_responses":True,
        "project_id":"00F3103E41FF3743A057C3B5313595B7",
        "object_id":"98EB31B54122FFB738E6E08A2F29421A"
}}

redis_con_d=redis_env_d["mstr_test"]


In [ ]:
i_redis_bi_analysis_test=open_redis_connection(redis_con_d=redis_env_d["mstr_test"])
i_redis_bi_analysis_dev=open_redis_connection(redis_con_d=redis_env_d["mstr_dev"])


In [ ]:
dossier_j=i_redis_bi_analysis_test.fetch_key_value("98EB31B54122FFB738E6E08A2F29421A")

get_dependency(i_redis_bi_analysis=i_redis_bi_analysis_dev
               ,object_j=dossier_j
               ,target_keys_l=["id", "key", "objectId"])



[{'id': '98EB31B54122FFB738E6E08A2F29421A',
  'name': 'Detailed data',
  'datasets': [{'name': 'Customer Detail report (Dashboard)',
    'id': 'C060EF484E1352B26D945CBAF191F2B8',
    'availableObjects': [{'id': '56225F324F17B61AEBD88CA256946C96',
      'name': 'Customer Address',
      'type': 'attribute',
      'forms': [{'id': '45C11FA478E745FEA08D781CEA190FE5',
        'name': 'Address',
        'dataType': 'varChar',
        'baseFormCategory': 'ID',
        'baseFormType': 'text'}]},
     {'id': '8D679D4211D3E4981000E787EC6DE8A4',
      'name': 'Item',
      'type': 'attribute',
      'forms': [{'id': '45C11FA478E745FEA08D781CEA190FE5',
        'name': 'ID',
        'dataType': 'integer',
        'baseFormCategory': 'ID',
        'baseFormType': 'number'}]},
     {'id': '6E069E4C11D3E4E41000E887EC6DE8A4',
      'name': 'Customer Age',
      'type': 'attribute',
      'forms': [{'id': '45C11FA478E745FEA08D781CEA190FE5',
        'name': 'ID',
        'dataType': 'integer',
        '

In [5]:
len(all_obj_def_d_l)
#child_obj_d_l=i_redis_bi_analysis.fetch_key_list(key_l=list(set(all_keys_l
#all_obj_def_d_l[2]

186